# Creating this notebook to manually fix issues in the DB as they come up!

## Fix 1: discipline strings
extra whitespace in the discipline values

In [1]:
import pandas as pd
import numpy as np
import sqlite3

from queries import get_race_data, get_point_total, get_races_list, audit_df, get_race_years

In [2]:
# Set the database path to a location with write permissions
db_path = '../race_league_results.db'

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect(db_path)
cursor = conn.cursor()


In [14]:
sql = """
UPDATE RaceResults
SET discipline = TRIM(discipline);
"""

cursor = conn.cursor()
cursor.execute(sql)
conn.commit()

In [15]:
sql = """
select discipline, COUNT(*)
from RaceResults
group by 1;
"""
df = pd.read_sql(sql, conn)

In [16]:
df

,discipline,COUNT(*)
0,None,16
1,SKI,3900
2,SNBD,701


## Fix Jeffrey Solomon's name...
already fixed the csv file.

In [82]:
sql = """
UPDATE RaceResults
SET racer_id = 'jeffreysolomon'
WHERE racer_id = 'jefferysolomon';
"""

cursor = conn.cursor()
cursor.execute(sql)
conn.commit()